In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_data = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


cuda


In [2]:
#Here is the implementation of MPL from PyTorch doc.
#We will use this model as an application of automatic differentiation.
class MPL(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#Here is the implementation of MPL from PyTorch doc.
#We will use this model as an application of automatic differentiation.
    
model = MPL().to(device)
print(model)

MPL(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): ReLU()
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): ReLU()
    (10): Linear(in_features=10, out_features=10, bias=True)
    (11): ReLU()
    (12): Linear(in_features=10, out_features=10, bias=True)
    (13): ReLU()
    (14): Linear(in_features=10, out_features=10, bias=True)
    (15): ReLU()
  )
)


In [3]:
def train_loop(data_loader, model, optimizer, loss, device):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)

        # Forward pass
        pred = model(data)
        loss_value = loss(pred, target)

        # Backward pass
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {batch_idx} \tLoss: {loss_value.item():.6f}")
def test_loop(data_loader, model, loss, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(data_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(data_loader.dataset)} ({100. * correct / len(data_loader.dataset):.0f}%)\n")
    
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 100 + 1):
    train_loop(train_loader, model, optimizer, loss_fn, device)
    test_loop(test_loader, model, loss_fn, device)



Train Epoch: 0 	Loss: 2.307140
Train Epoch: 100 	Loss: 2.112789
Train Epoch: 200 	Loss: 1.942459
Train Epoch: 300 	Loss: 1.850685
Train Epoch: 400 	Loss: 1.780811
Train Epoch: 500 	Loss: 1.633669
Train Epoch: 600 	Loss: 1.582213
Train Epoch: 700 	Loss: 1.520120
Train Epoch: 800 	Loss: 1.345459
Train Epoch: 900 	Loss: 1.659579

Test set: Average loss: 0.0229, Accuracy: 4819/10000 (48%)

Train Epoch: 0 	Loss: 1.446252
Train Epoch: 100 	Loss: 1.268439
Train Epoch: 200 	Loss: 1.483118
Train Epoch: 300 	Loss: 1.392259
Train Epoch: 400 	Loss: 1.694363
Train Epoch: 500 	Loss: 1.337506
Train Epoch: 600 	Loss: 1.584475
Train Epoch: 700 	Loss: 1.489948
Train Epoch: 800 	Loss: 1.484967
Train Epoch: 900 	Loss: 1.612112

Test set: Average loss: 0.0226, Accuracy: 4844/10000 (48%)

Train Epoch: 0 	Loss: 1.341525
Train Epoch: 100 	Loss: 1.239361
Train Epoch: 200 	Loss: 1.231942
Train Epoch: 300 	Loss: 1.595712
Train Epoch: 400 	Loss: 1.550941
Train Epoch: 500 	Loss: 1.316496
Train Epoch: 600 	Loss: 1.